# Expectation Values

In MatchCake, we can compute expectation values of Hamiltonians of the form

$$
\mathcal{H} = \sum\limits_{k=0}^{K-1} \alpha_k P_k,
$$

where each $P_k$ is a Pauli word that can be decomposed into Gaussian Majorana operators from the set

$$
\{Z_{i}I_{i+1}, X_{i}X_{i+1}, Y_{i}Y_{i+1}, Y_{i}X_{i+1}, X_{i}Y_{i+1}, I_{i}Z_{i+1} \}
$$

where $i$ denoting the index of the first wire on which the Pauli word acts.

The expectation value of the Hamiltonian is given by

$$
\langle \mathcal{H} \rangle = \langle \psi_{\text{prod}}| U^\dagger \mathcal{H} U | \psi_{\text{prod}} \rangle
$$

where $U$ is a matchcircuit and $| \psi_{\text{prod}} \rangle$ is a product state. Note that MatchCake currently supports only computational basis product states. Ongoing work aims to extend this functionality to arbitrary product states; see [the issue 54](https://github.com/MatchCake/MatchCake/issues/54) for further details.

In this tutorial, we demonstrate how to compute such expectation values using MatchCake.


In [1]:
import matchcake as mc
import pennylane as qml
import numpy as np

We now construct a Hamiltonian with arbitrary coefficients for each Pauli term.

In [2]:
hamiltonian = qml.Hamiltonian(
    [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
    [
        qml.X(0) @ qml.X(1),
        qml.Y(1) @ qml.Y(2),
        qml.Z(0) @ qml.Z(1),
        qml.Y(1) @ qml.X(2),
        qml.X(0) @ qml.Y(1),
        qml.Z(1) @ qml.I(2),
        qml.I(1) @ qml.Z(2),
    ]
)

We then prepare the initial state.

In [3]:
state_prep_op = qml.BasisState([1, 0, 1], wires=hamiltonian.wires)

We now generate a random single-particle transition matrix, convert it into a unitary operator, and simulate the resulting circuit using the Non-Interacting Fermionic Device.

In [4]:
sptm = mc.operations.SingleParticleTransitionMatrixOperation.random(wires=hamiltonian.wires, seed=0)
U = sptm.to_qubit_unitary()

@qml.qnode(mc.NIFDevice(3))
def circuit():
    state_prep_op.queue()
    sptm.queue()
    return qml.expval(hamiltonian)

We evaluate the expectation value of the Hamiltonian.

In [5]:
expval = circuit()
print(expval)

tensor(-0.7878, dtype=torch.float64)


Finally, we compare our result with that obtained from the state-vector simulator to validate our computation.

In [6]:
@qml.qnode(qml.device("default.qubit", wires=3))
def svs_circuit():
    state_prep_op.queue()
    U.queue()
    return qml.expval(hamiltonian)

svs_expval = svs_circuit()
print(svs_expval)
np.testing.assert_allclose(expval, svs_expval, atol=1e-5)

-0.7878426251216495


------------------------------------------------------------------------